<center>
<p><img src="https://www.gob.mx/cms/uploads/image/file/179499/outstanding_quienes-somos.jpg" width="300">
</p>



# Curso *Machine Learning con uso de pandas, scikit learn y libretas jupyter*

# Uso de `scikit-learn` y `skforcast` para predicción del consumo de energía eléctrica. 


<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="80">
</p>
</center>



En este documento se muestra un ejemplo de cómo utilizar métodos de *forecasting* para predecir la demanda eléctrica en Hermosillo a nivel horario. Vamos a utilizar un conjunto de datos que puso generosamente a nuestra disposición [Hector Alberto Gutierrez Ibarra](hector.gutierrez@cenace.gob.mx) de la gerencia noroeste.

Para ello, se hace uso de [`skforecast`](https://github.com/JoaquinAmatRodrigo/skforecast), una librería de Python que permite adaptar cualquier regresor de `scikit-learn` a problemas de forecasting. Esta libreta retoma en gran parte el trabajo de [Joaquín Amat](https://www.cienciadedatos.net/index.html) en su muy interesante post sobre [Predicción (forecasting) de la demanda eléctrica con Python](https://www.cienciadedatos.net/documentos/py29-forecasting-demanda-energia-electrica-python.html).

iniciemos instalando las bibliotecas que no vienen por *default* en Colab.


In [ ]:
!pip install skforecast

## Cargando el conjunto de datos y limpiandolo

Vamos a repetir algunos pasos que ya hicimos en una libreta pasada en el procesamiento de la información. Al mismo tiempo, vamos a agregar una información nueva, la información sobre los días festivos. Tambien vamos a incluir una columna sólo con la fecha, para diferenciar de la hora.

Un problema importante para poder aplicar los métodos de autoregresión que vamos a ver más adelante, es que es necesario asegurarse que los datos se encuentren separados a intervalos de tiempo regular (en nuestro caso a 1 hora).

De no ser así es necesario realizar un procedimiento de imputación de datos, y asignarles, ya sea un valor numérico, ya sea marcar explicitamente como un dato perdido. Para eso vamos a usar la función `pd.asfreq` que nos asegura una serie a intervalos regulares. Siempre hay que estar muy pendientes para evitar usar datos futuros en el pasado.

Vamos a los datos:


In [ ]:
import pandas as pd

url = "https://github.com/juliowaissman/curso-ml-cenace/raw/main/datos/caso_zc_hmo.csv.zip"
df = pd.read_csv(url)

df['Date'] = pd.to_datetime(df.Date, format="%d/%m/%Y %H:%M")
df.index = df.Date
df = df.asfreq('H', method='pad')
df.rename(
    columns={
        'Date': 'Fecha',
        'Demand': 'Demanda',
        'Temperature': 'Temperatura',
        'PrecipIntensity': 'Precipitación',
        'Humidity': 'Humedad',
        'WinSpeed': 'VelocidadViento',
    },
    inplace=True
)
df['Dia'] = df.Fecha.dt.day_of_week
df['Mes'] = df.Fecha.dt.month
df

Ahora vamos a agregar una columna con los días festivos de Mexico gracias a la biblioteca `holidays`.


In [ ]:
!pip install holidays

In [ ]:
import holidays

festivos = list(holidays.MEX(years=[2016, 2017, 2018, 2019, 2020, 2021]).keys())
df['Festivos'] = 1
df.Festivos.where(df.Fecha.dt.date.isin(festivos), 0, inplace=True)
df.Festivos.plot()
df

Ahora veamos si hay diferencias en el consumo de energía entre los días festivos y los domingos y los otros días

In [ ]:
df[['Demanda']].groupby(df.Festivos).boxplot(subplots=False)

## Multi-Step Time Series Forecasting

Cuando se trabaja con series temporales, raramente se quiere predecir solo el siguiente elemento de la serie $x_{t+1}$, sino todo un intervalo futuro o un punto alejado en el tiempo $x_{t+n}$. A cada paso de predicción se le conoce como *step*. Existen varias estrategias que permiten generar este tipo de predicciones múltiples.

### Recursive multi-step forecasting 

Dado que, para predecir $x_{t+n}$ se necesita $x_{t+n-1}$, pero $x_{t+n-1}$ se desconoce, es necesario hacer predicciones recursivas en las que, cada nueva predicción, se basa en la predicción anterior. A este proceso se le conoce como *recursive forecasting* o *recursive multi-step forecasting*.

![](https://www.cienciadedatos.net/images/forecasting_multi-step.gif)

La principal adaptación que se necesita hacer para aplicar modelos de *scikit-learn* a problemas de *recursive multi-step forecasting* es transformar la serie temporal en un matriz en la que, cada valor, está asociado a la ventana temporal (lags) que le preceden. Esta estrategia de forecasting pueden generarse fácilmente con las clases `ForecasterAutoreg` y `ForecasterAutoregCustom` de la librería `skforecast`.

![](https://www.cienciadedatos.net/images/transform_timeseries.gif)



### Direct multi-step forecasting 

El método *direct multi-step forecasting* consiste en entrenar un modelo distinto para cada *step*. Por ejemplo, si se quieren predecir los siguientes 5 valores de una serie temporal, se entrenan 5 modelos distintos, uno para cada step. Como resultado, las predicciones son independientes unas de otras.

La principal complejidad de esta aproximación consiste en generar correctamente las matrices de entrenamiento para cada modelos. Todo este proceso está automatizado en la clase `ForecasterAutoregMultiOutput` de la librería `skforecast`. En el siguiente esquema se muestra el proceso para un caso en el que se dispone de la variable respuesta y dos variables exógenas.

![](https://www.cienciadedatos.net/images/diagram_skforecast_multioutput.png)






## Preparando los datos para el aprendizaje

En series de tiempo, se acostumbra entrenar con la mayor parte de los datos y utilizar sólo los últimos datos para prueba. En nuestro caso vamos a ser ambiciosos/inocentes, y vamos a utilizar como datos de validación todos los datos del 2021, y todos los datos anteriores para el entrenamiento.


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')


df_train = df[df.index.year < 2021]
df_val = df[df.index.year == 2021]

fig, ax = plt.subplots(figsize=(12, 4))
df_train.Demanda.plot(ax=ax, label='entrenamiento', linewidth=0.5)
df_val.Demanda.plot(ax=ax, label='validación', linewidth=0.5)
ax.legend();

## Modelo pseudo autoregresivo recursivo con regresores de *scikit-learn*

Vamos a iniciar el uso de `skforcaster` creando y entrenando un modelo autorregresivo recursivo (`ForecasterAutoreg`) a partir de un modelo de regresión lineal con penalización `Ridge` y una ventana temporal de 24 lags. Esto último significa que, para cada predicción, se utilizan como predictores la demanda de las 24 horas anteriores.

### Entrenamiento del Forecaster

Vamos a ir cargando las librerías conforme las vayamos necesitando en el afan de ejemplificar mejor el uso de las herramientas. 

In [ ]:
from sklearn.linear_model import Ridge
from skforecast.ForecasterAutoreg import ForecasterAutoreg

forecaster = ForecasterAutoreg(
  regressor= Ridge(normalize=True),
  lags= 24
)

forecaster.fit(y=df_train.Demanda)
forecaster

### Predicción (*backtest*)

Para evaluar el comportamiento del modelo entrenado, vamos a evalúa el comportamiento que habría tenido el modelo si  después, a las 23:59 de cada día, se predijesen las 24 horas siguientes. Toda la serie completa (datos de entrenamiento y prueba)

A este tipo de evaluación se le conoce como *backtesting*, y puede aplicarse fácilmente con la función `backtesting_forecaster()`. Esta función devuelve, además de las predicciones, una métrica de error.

In [ ]:
from skforecast.model_selection import backtesting_forecaster

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


metrica, predicciones = backtesting_forecaster(
    forecaster= forecaster,
    y= df.Demanda,
    initial_train_size = len(df_train.Demanda),
    steps= 24,
    metric= 'mean_absolute_error',
    verbose= True
)

# Se añade el índice temporal a las predicciones
predicciones = pd.Series(data=predicciones, index=df_val.index)

print(f"Error MAP = {metrica}")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
df_val.loc[predicciones.index, 'Demanda'].plot(ax=ax, linewidth=2, label='validación')
predicciones.plot(ax=ax, linewidth=2, label='predicción')
ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

y como es dificil ver algo, vamos haciendo un zoom por semana del año:

In [ ]:
semana = 15

fig, ax = plt.subplots(figsize=(15, 7))

df_val.loc[
    predicciones.index.isocalendar().week == semana, 
    'Demanda'
].plot(ax=ax, linewidth=2, label='validación')

predicciones[
    predicciones.index.isocalendar().week == semana
].plot(ax=ax, linewidth=2, label='predicción')

ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

### Optimización de hiperparámetros (tuning)


En el objeto ForecasterAutoreg entrenado, se han utilizado los primeros 24 lags y un modelo Ridge con los hiperparámetros por defecto. Sin embargo, no hay ninguna razón por la que estos valores sean los más adecuados.

Con el objetivo de identificar la mejor combinación de lags e hiperparámetros, se recurre a un *Grid Search* con validación por *backtesting*. Este proceso consiste en entrenar un modelo con cada combinación de hiperparámetros y lags, y evaluar su capacidad predictiva mediante *backtesting*. 

In [ ]:
import numpy as np
from skforecast.model_selection import grid_search_forecaster

forecaster = ForecasterAutoreg(
    regressor= Ridge(normalize=True),
    lags= 24 # Este valor será remplazado en el grid search
)

# Hiperparámetros del regresor
param_grid = {'alpha': np.logspace(-3, 3, 10)}

# Lags utilizados como predictores
lags_grid = [5, 24, [1, 2, 3, 23, 24, 25, 47, 48, 49]]

resultados_grid = grid_search_forecaster(
    forecaster= forecaster,
    y= df_train.Demanda,
    param_grid= param_grid,
    lags_grid= lags_grid,
    steps= 24,
    metric= 'mean_absolute_error',
    method= 'backtesting',
    initial_train_size = len(df_train.Demanda[df_train.index.year < 2020]),
    return_best= True,
    verbose= False
)

In [ ]:
resultados_grid

Los mejores resultados se obtienen si se utilizan los lags [1, 2, 3, 23, 24, 25, 47, 48, 49] y una $\alpha = 0.001$ para el método de regresión `Ridge`. 

Al indicar `return_best = True` en la función `grid_search_forecaster()`, al final del proceso, se reentrena automáticamente el objeto forecaster con la mejor configuración encontrada y el set de datos completo.



In [ ]:
forecaster

Revisemos si este algoritmo mejora respecto al anterior que no usamos una búsqueda ávida de los mejores parámetros

In [ ]:
metrica, predicciones = backtesting_forecaster(
    forecaster= forecaster,
    y= df.Demanda,
    initial_train_size = len(df_train.Demanda),
    steps= 24,
    metric= 'mean_absolute_error',
    verbose= True
)

# Se añade el índice temporal a las predicciones
predicciones = pd.Series(data=predicciones, index=df_val.index)

print(f"Error MAP = {metrica}")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
df_val.loc[predicciones.index, 'Demanda'].plot(ax=ax, linewidth=2, label='validación')
predicciones.plot(ax=ax, linewidth=2, label='predicción')
ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

In [ ]:
semana = 15

fig, ax = plt.subplots(figsize=(15, 7))

df_val.loc[
    predicciones.index.isocalendar().week == semana, 
    'Demanda'
].plot(ax=ax, linewidth=2, label='validación')

predicciones[
    predicciones.index.isocalendar().week == semana
].plot(ax=ax, linewidth=2, label='predicción')

ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

Pues al parecer el error se redujo a la mitad, auque todavía hay bastante espacio para mejorar.

### Intervalos de predicción


Un intervalo de predicción define el intervalo dentro del cual es de esperar que se encuentre el verdadero valor de  𝑦
y
 con una determinada probabilidad.

Rob J Hyndman y George Athanasopoulos, listan en su libro [Forecasting: Principles and Practice](https://otexts.com/fpp2/) mútiples formas de [estimar intervalos de predicción](https://otexts.com/fpp2/prediction-intervals.html), la mayoría los cuales requieren que los resudios (errores) del modelo se distribuyan de forma normal. Cuando no se puede asumir esta propiedad, se puede recurrir a *bootstrapping*, que *solo* asume que los residuos no están correlacionados. Este es el método utilizado en la librería `skforecast` para los modelos de tipo `ForecasterAutoreg` y `ForecasterAutoregCustom`. 

In [ ]:
from skforecast.model_selection import backtesting_forecaster_intervals

metric, predictions = backtesting_forecaster_intervals(
    forecaster= forecaster,
    y= df.Demanda,
    initial_train_size = len(df_train.Demanda),
    steps= 24,
    metric= 'mean_absolute_error',
    interval= [10, 90],
    n_boot= 500,
    in_sample_residuals= True,
    verbose= True
)

print('Métrica backtesting:', metric)

# Se añade índice datetime
predictions = pd.DataFrame(data=predictions, index=df_val.index)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
df_val.loc[predictions.index, 'Demanda'].plot(
    ax=ax, lw=2, label='validación' 
)
predictions.iloc[:, 0].plot(
    ax=ax, lw=2, label='predicción' 
)
ax.fill_between(
    predictions.index,
    predictions.iloc[:, 1],
    predictions.iloc[:, 2],
    alpha = 0.2,
    color = 'red',
    label = 'Intervalo predicción' 
)
ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

Y de nuevo, mejor lo vemos por semana

In [ ]:
semana = 15

fig, ax = plt.subplots(figsize=(15, 7))
df_val.loc[predictions.index.isocalendar().week == semana, 'Demanda'].plot(
    ax=ax, lw=2, label='validación' 
)
predictions.loc[predictions.index.isocalendar().week == semana, 0].plot(
    ax=ax, lw=2, label='predicción' 
)
ax.fill_between(
    predictions[predictions.index.isocalendar().week == semana].index,
    predictions.loc[predictions.index.isocalendar().week == semana, 1],
    predictions.loc[predictions.index.isocalendar().week == semana, 2],
    alpha = 0.2,
    color = 'red',
    label = 'Intervalo predicción' 
)
ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

¿Qué porcentaje de tiempo la demanda real estuvo dentro del intervalo de confianza de la predicción? Este es un buen indicador de la bondad del modelo.


In [ ]:
dentro_intervalo = np.where(
  (df_val.Demanda >= predictions.iloc[:, 1]) & \
  (df_val.Demanda <= predictions.iloc[:, 2]),
  True,
  False
)

cobertura = dentro_intervalo.mean()
print(f"Cobertura del intervalo predicho: {100 * cobertura:.2f}%")

El intervalo predicho tiene una cobertura inferior a la que cabría esperar (80%). Las razones se deben buscar en los días donde la predicción estuvo fuera de rango. Las razones pueden ser por ser días atípicos (domingos o festivos), o por situaciones meteorológicas entre otras.

### Predicción diaria anticipada

En el apartado anterior, se evaluó el modelo asumiendo que las predicciones del día siguiente se ejecutan justo al final del día anterior. En la práctica, esto no resulta muy útil ya que, para las primeras horas del día, apenas se dispone de anticipación.

Supóngase ahora que, para poder tener suficiente margen de acción, a las 11:00 horas de cada día se tienen que generar las predicciones del día siguiente. Es decir, a las 11:00 del dia $D$ se tienen que predecir las horas [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23] de ese mismo día, y las horas [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23] del día $D+1$. Esto implica que se tienen que predecir un total de 36 horas a futuro.

El proceso de *backtesting* adaptado a este escenario es:

1. A las 11:00h del primer día del conjunto de test, se predicen las 36 siguientes horas (las 12 horas que quedan del día más las 24 horas de el día siguiente).

2. Se almacenan solo las predicciones del día siguiente, es decir, de la posición 12 en adelante.

3. Se añaden los datos de test hasta las 11:00 del día siguiente.

4. Se repite el proceso.

De esta forma, a las 11:00h de cada día, el modelo tiene acceso a los valores reales de demanda registrados hasta ese momento.

Este proceso puede realizarse *fácilmente* con el método `predict()` de un objeto `ForecasterAutoreg`. Si no se le indica nada, la predicción se inicia después del último valor de entrenamiento, pero, si se le especifica el argumento `last_window`, utiliza estos valores como punto de partida.

Anexamos a contunuación una función que no está dentro de ´skforcast` pero que no tardará en ser agregada a la biblioteca.

In [86]:
def backtest_predict_next_24h(
    forecaster, y, hour_init_prediction, exog=None, verbose=False):
    '''
    Backtest ForecasterAutoreg object when predicting 24 hours of day D+1
    statring at specific hour of day D.
    
    Parameters
    ----------
    forecaster : ForecasterAutoreg 
        ForecasterAutoreg object already trained.
        
    y : pd.Series with datetime index sorted
        Test time series values. 
        
    exog : pd.Series or pd.Dataframe with datetime index sorted
        Test values of exogen variable. 
    
    hour_init_prediction: int 
        Hour of day D to start predictions of day D+1.


    Returns 
    -------
    predictions: pd.Series
        Value of predictions.

    '''
    
    y = y.sort_index()
    if exog is not None:
        exog = exog.sort_index()
        
    dummy_steps = 24 - (hour_init_prediction + 1)
    steps = dummy_steps + 24
    
    # First position of `hour_init_prediction` in the series where there is enough
    # previous window to calculate lags.
    for datetime in y.index[y.index.hour == hour_init_prediction]:
        if len(y[:datetime]) >= len(forecaster.last_window):
            datetime_init_backtest = datetime
            print(f"Backtesting starts at day: {datetime_init_backtest}")
            break
    
    days_backtest = np.unique(y[datetime_init_backtest:].index.date)
    days_backtest = pd.to_datetime(days_backtest)
    days_backtest = days_backtest[1:]
    print(f"Days predicted in the backtesting: {days_backtest.strftime('%Y-%m-%d').values}")
    print('')
    backtest_predictions = []
    
    for i, day in enumerate(days_backtest):        
        # Start and end of the last window used to create the lags
        end_window = (day - pd.Timedelta(1, unit='day')).replace(hour=hour_init_prediction)
        start_window = end_window - pd.Timedelta(forecaster.max_lag, unit='hour')
        last_window = y.loc[start_window:end_window]
               
        if exog is None:
            if verbose:
                print(f"Forecasting day {day.strftime('%Y-%m-%d')}")
                print(f"Using window from {start_window} to {end_window}")
                
            pred = forecaster.predict(steps=steps, last_window=last_window)
            
        else:
            start_exog_window = end_window + pd.Timedelta(1, unit='hour')
            end_exog_window   = end_window + pd.Timedelta(steps, unit='hour')
            exog_window = exog.loc[start_exog_window:end_exog_window]
            exog_window = exog_window.to_numpy()
            
            if verbose:
                print(f"Forecasting day {day.strftime('%Y-%m-%d')}")
                print(f"    Using window from {start_window} to {end_window}")
                print(f"    Using exogen variable from {start_exog_window} to {end_exog_window}")
            
            pred = forecaster.predict(steps=steps, last_window=last_window, exog=exog_window)

        # Only store predictions of day D+1
        pred = pred[dummy_steps:]
        backtest_predictions.append(pred)
    
    backtest_predictions = np.concatenate(backtest_predictions)
    # Add datetime index
    backtest_predictions = pd.Series(
                             data  = backtest_predictions,
                             index = pd.date_range(
                                        start = days_backtest[0],
                                        end   = days_backtest[-1].replace(hour=23),
                                        freq  = 'h'
                                    )
                           )
    
    return backtest_predictions

y ahora la vamos a usar (como se puede usar en otros problemas con el mismo tipo de problemas)

In [ ]:
predicciones = backtest_predict_next_24h(
  forecaster= forecaster,
  y= df_val.Demanda,
  hour_init_prediction= 11,
  verbose= False
)

error = mean_absolute_error(
    y_true= df_val.loc[predicciones.index, 'Demanda'],
    y_pred= predicciones
)
print(f"Error de backtest (MAP): {error}")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
df_val.loc[predicciones.index, 'Demanda'].plot(ax=ax, lw=2, label='validación')
predicciones.plot(ax=ax, lw=2, label='predicción')
ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

In [ ]:
semana = 15

fig, ax = plt.subplots(figsize=(15, 7))

df_val.loc[ 
    df_val.index.isocalendar().week == semana, 'Demanda'
].plot(ax=ax, linewidth=2, label='validación')

predicciones[
    predicciones.index.isocalendar().week == semana
].plot(ax=ax, linewidth=2, label='predicción')

ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

Como era de esperar, al aumentar el horizonte de predicción de 24 a 36 horas, también lo hace el error de las predicciones.

### Importancia predictores

Dado que el objeto `ForecasterAutoreg` utiliza modelos *scikit-learn*, una vez entrenado, se puede acceder a la importancia de los predictores. Cuando el regresor empleado es un `LinearRegression()`, `Lasso()` o `Ridge()`, la importancia queda reflejada en los coeficientes del modelo, que se obtienen con el método `get_coef()`. En regresores  `GradientBoostingRegressor()` o `RandomForestRegressor()`, la importancia de los predictores está basada en la reducción de impureza y es accesible mediante el método `get_feature_importances()`.

In [ ]:
importancia = pd.DataFrame({
    'lag': forecaster.lags, 
    'coeficiente': forecaster.get_coef()
})

importancia

## Forecasting con variables exógenas

En el ejemplo anterior, se han utilizado como predictores únicamente lags de la propia variable objetivo. En ciertos escenarios, es posible disponer de información sobre otras variables, cuyo valor a futuro se conoce, y que pueden servir como predictores adicionales en el modelo. Algunos ejemplos típicos son:

- Festivos (local, nacional...)
- Mes del año
- Día de la semana
- Hora del día

En este caso de uso, el análisis gráfico mostraba evidencias de que, los días festivos, la demanda es menor. Si un día es festivo o no, puede saberse a futuro, por lo que se puede emplear como variable exógena. Véase cómo afecta al modelo si se incluye como predictor la variable `Festivo disponible en el dataframe.

### Entrenamiento del Forecaster

El uso de variables exógenas es directo de los mñetodos que ya hemos utilizado



In [ ]:
forecaster = ForecasterAutoreg(
  regressor= Ridge(alpha=0.001, normalize=True),
  lags= [1, 2, 3, 23, 24, 25, 47, 48, 49],
)
forecaster.fit(y=df_train.Demanda, exog=df_train.Festivos)
forecaster

### Predicción diaria anticipada


Se repite de nuevo el proceso de backtesting en el que, a las 11:00 horas de cada día, se tienen que obtener las predicciones del día siguiente. Esta vez, incluyendo como predictor si el día es festivo o no.

In [ ]:
predicciones = backtest_predict_next_24h(
    forecaster= forecaster,
    y= df_val.Demanda,
    exog= df_val.Festivos,
    hour_init_prediction= 11,
    verbose= False
)

error = mean_absolute_error(
    y_true= df_val.loc[predicciones.index, 'Demanda'],
    y_pred= predicciones
)
print(f"Error de backtest (MAP): {error}")

Los días festivos al parecer no tienen gran influencia en la demanda de energía, al menos en el 2021.

Veamos que pasa si agregamos como variables exógenas el día de la semana, el mes del año y la hora del día.

In [117]:
# Agregamos hora que no lo habíamos hecho antes
df['Hora'] = df.Fecha.dt.hour

# One hot encoding de las variables mes, hora y dia
df=pd.get_dummies(df, columns=['Hora', 'Dia', 'Mes'])

In [ ]:
df_train = df[df.index.year < 2021]
df_val = df[df.index.year == 2021]
df.head(3)

Ahora si estamos en condiciones de aplicar el entrenamiento con las variables exógenas

In [120]:
forecaster = ForecasterAutoreg(
    regressor= Ridge(alpha=0.001, normalize=True),
    lags= [1, 2, 3, 23, 24, 25, 47, 48, 49]
)

exog = [column for column in df.columns 
        if column.startswith(('Dia', 'Hora', 'Mes', 'Festivos'))]

forecaster.fit(y=df_train.Demanda, exog=df_train[exog].values)


Y ahora podemos probar la bondad del modelo entrenado

In [ ]:
predicciones = backtest_predict_next_24h(
    forecaster= forecaster,
    y= df_val.Demanda,
    exog= df_val[exog],
    hour_init_prediction= 11,
    verbose= False
)

error = mean_absolute_error(
  y_true = df_val.loc[predicciones.index, 'Demanda'],
  y_pred = predicciones
)
print(f"Error de backtest (MAP): {error}")

y como vemos hay una reducción importante del error (MAP).

In [ ]:
semana = 15

fig, ax = plt.subplots(figsize=(15, 7))

df_val.loc[ 
    df_val.index.isocalendar().week == semana, 'Demanda'
].plot(ax=ax, linewidth=2, label='validación')

predicciones[
    predicciones.index.isocalendar().week == semana
].plot(ax=ax, linewidth=2, label='predicción')

ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

### Incluir temperatura como predictor

Dado que en el set de datos también se dispone de la temperatura, y esta está relacionada con la demanda, podría ser tentador incorporarla como predictor. Sin embargo, esta aproximación no sería correcta ya que, la temperatura no se conoce a futuro. Sí es posible utilizar la previsión de tempratura como un predictor del modelo pero, en tal caso, durante el entrenamiento habría que utilizar la previsión que había en ese momento, no la temperatura real. 

## Modelo direct multi-step

Los modelos `ForecasterAutoreg` siguen una estrategia de predicción recursiva en la que, cada nueva predicción, se basa en la predicción anterior. Una alternativa es entrenar un modelo para cada uno de los steps que se desea predecir, lo que se conoce como *direct multi-step forecasting*. Si bien es computacionalmente más costosa que la recursiva, puesto que requiere entrenar múltiples modelos, en algunos escenarios, consigue mejores resultados. Este tipo de modelos pueden obtenerse con la clase *ForecasterAutoregMultiOutput* y pueden incluir también una o múltiples variables exógenas.

### Entrenamiento y tuning del Forecaster

A diferencia de cuando se utiliza `ForecasterAutoreg`, en los modelos de tipo `ForecasterAutoregMultiOutput` hay que indicar, en el momento de su creación, el número de steps que se quieren predecir. Esto significa que, el número de predicciones obtenidas al ejecutar el método `predict()`, es siempre el mismo.

In [ ]:
from sklearn.linear_model import ElasticNet
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput


forecaster = ForecasterAutoregMultiOutput(
    regressor= ElasticNet(),
    steps= 36,
    lags= 24 # Este valor será remplazado en el grid search
)

# Hiperparámetros del regresor
param_grid = {'alpha': [0.01, 0.1, 1]}

# Lags utilizados como predictores
lags_grid = [[1, 2, 3, 23, 24]]

exog = [column for column in df.columns 
        if column.startswith(('Dia', 'Hora', 'Mes', 'Festivo'))]

resultados_grid = grid_search_forecaster(
    forecaster= forecaster,
    y= df_train.Demanda,
    exog= df_train[exog].values,
    param_grid= param_grid,
    lags_grid= lags_grid,
    steps= 36,
    metric= 'mean_absolute_error',
    method= 'backtesting',
    initial_train_size= len(df_train.Demanda[df_train.index.year < 2020]),
    return_best= True,
    verbose= False
)

Y el error se calcula de la misma forma que con el forcaster anterior.

In [ ]:
predicciones = backtest_predict_next_24h(
    forecaster= forecaster,
    y= df_val.Demanda,
    exog= df_val[exog],
    hour_init_prediction= 11,
    verbose= False
)

error = mean_absolute_error(
  y_true = df_val.loc[predicciones.index, 'Demanda'],
  y_pred = predicciones
)
print(f"Error de backtest (MAP): {error}")

In [ ]:
semana = 15

fig, ax = plt.subplots(figsize=(15, 7))

df_val.loc[ 
    df_val.index.isocalendar().week == semana, 'Demanda'
].plot(ax=ax, linewidth=2, label='validación')

predicciones[
    predicciones.index.isocalendar().week == semana
].plot(ax=ax, linewidth=2, label='predicción')

ax.set_title('Predicción vs demanda real')
ax.legend()
plt.show()

## Lo que falta

Ahora hay que practicar y realizar un ejercicio, reutilizando todas las herramientas que hemos visto al momento. 

Si deseas practicar con los mismos datos hay muchas cosas que hacer:

1. Explorar entre las opciones de *lags* las que, a partir de su conocimiento experto suenen como las más prometedoras.
2. Seleccionar otro método de regresión que pueda ser interesante, de acuerdo a la problemática y dinámica que se conoce de los datos. Explorar los parámetros posibles con una búsqueda ávida.
3. Con un modelo seleccionado y entrenado, verificar cuales son los días en los cuales la predicción se distancia de la real fuera del intervalo de confianza. Revisar esos días y tratar de encontrar las causas en los datos existentes.